# Angular Component Generator

This notebook leverages an embeddings model to analyze the code structure of this application, and generates a new Angular component with a foundational model that adheres to the proprietary tooling and best practices observed in the existing codebase. 

By using LangChain to manage prompt templates and employing Retrieval-Augmented Generation (RAG) to provide context around embeddings, we ensure that the generated code is both relevant and aligned with the existing codebase. This process demonstrates the capability of fine-tuning a model to follow specific development conventions and incorporate them into the generated code.

To run this notebook, ensure you have access to the following models within Bedrock:
- **Titan Text G1 - Premier**
- **Titan Embeddings G1 - Text**

## Step 1: Environment Setup

Initialize all necessary libraries and create an instance of the AWS BedRock boto3 client. This step ensures that the environment is properly configured to interact with the Bedrock service.

In [ ]:
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Step 2: Load and Analyze Angular Components

Load the Angular components from subdirectories within the `app` folder and use the embeddings model to analyze their structure. This step involves reading the files, creating embeddings, and storing them in a vector store for later retrieval.

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os

def load_all_files_recursive(directory):
    documents = []
    subfolders = set()  # Set to store unique subfolder names
    for root, dirs, files in os.walk(directory):
        # Skip hidden subfolders
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        for file in files:
            filepath = os.path.join(root, file)
            try:
                loader = TextLoader(filepath, encoding="utf-8")
                loaded_docs = loader.load()
                # Filter out empty documents
                non_empty_docs = [doc for doc in loaded_docs if doc.page_content.strip()]
                documents.extend(non_empty_docs)
                print(f"Parsed file {filepath}")
            except UnicodeDecodeError:
                print(f"Skipping binary file or file with unknown encoding: {filepath}")
            except Exception as e:
                print(f"Error loading file {filepath}: {e}")

        # Add subfolder names to the set
        for subfolder in dirs:
            subfolders.add(os.path.relpath(os.path.join(root, subfolder), directory))
    return documents, ", ".join(sorted(subfolders))

# Load ALL files recursively
documents, subfolders = load_all_files_recursive("../src/app")
print(f"Loaded {len(documents)} documents.")

# Create embeddings
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

# Create vector store
vectorstore_faiss = FAISS.from_documents(documents, embedding=br_embeddings)
print("Vector store created.")

# Step 3: Create Component Knowledge Base
Add guidance around how to create a component to the vector DB.

In [ ]:
comp_struct = (
    "Each component must have 5 files:\n"
    "1. <name>.component.ts - containing the primary component code,\n"
    "2. <name>.config.ts - containing the config data,\n"
    "3. <name>.component.html - containing the HTML markup,\n"
    "4. <name>.component.css - stylesheets,\n"
    "5. <name>.component.spec.ts - unit tests.\n"
    "Analyze the files within the subfolders: " + subfolders + "\n"
    "And use them as examples on how each file is referenced from another.\n"
    "Strictly follow the following response template within ```: \n"
    "```\n"
    "/*-- <filename> --*/\n"
    "<generated code>\n"
    "```\n"
    "Do not respond back with any explanation in the beginning or end. "
    "Stop responding as soon as the last line of the code is output."
)

metadata = [{"guidelines": "Component structure"}]

vectorstore_faiss.add_texts([comp_struct], metadatas=metadata)
print("Component knowledge loaded.")

## Step 4: Create a ChatUX to Generate Code

This step defines the `ChatUX` class, which uses IPWidgets to create a user-friendly interface for inputting the component name and displaying the generated code. This utilizes a RAG chain and a LangChain tool to retrieve the business intent from a JIRA ticket.

In [ ]:
import ipywidgets as ipw
from IPython.display import display
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock
from langchain.agents import Tool

# Create a JIRA lookup stub
def jira_lookup(jira_id: str) -> str:
    jira_data = {
        "ABC-123": "The component should display 'Hello World :)'",
        "DEF-456": "The component should display 'Goodbye World :('",
    }
    return jira_data.get(jira_id, "ERRNOTFOUND")

# Define the JIRA lookup tool
jira_lookup_tool = Tool(
    name="jira_lookup",
    func=jira_lookup,
    description="Useful for when you need to lookup information about a JIRA "
                "ticket. You should only call this function with a JIRA "
                "ticket ID, e.g. 'ABC-123'. "
                "The input to this tool should be a string that is a valid "
                "JIRA ticket ID"
)

# Define ChatUX class
class ChatUX:
    """ A chat UX using IPWidgets """
    def __init__(self, rag_chain, prompt_template, jira_lookup_tool):
        self.rag_chain = rag_chain
        self.prompt_template = prompt_template
        self.jira_lookup_tool = jira_lookup_tool
        self.name = None
        self.btn = None
        self.out = ipw.Output()

    def display_box(self):
        with self.out:
            self.name = ipw.Text(placeholder='Enter component name')
            self.ticket_id = ipw.Text(placeholder='Enter JIRA ticket ID')
            self.btn = ipw.Button(description="Create")
            self.btn.on_click(self.chat)
            display(ipw.Box(children=(self.name, self.ticket_id, self.btn)))

    def start_chat(self):
        print("Welcome to the component generator!")
        self.display_box()
        display(self.out)

    def chat(self, _):
        with self.out:
            if len(self.name.value) > 0 and len(self.ticket_id.value) > 0:
                jira_ticket_data = self.jira_lookup_tool.invoke(self.ticket_id.value)

                if jira_ticket_data != "ERRNOTFOUND":
                    print("JIRA Ticket data: ", jira_ticket_data)
                    formatted_prompt = self.prompt_template.format(self.name.value, jira_ticket_data)

                    thinking = ipw.Label(value="Generating...")
                    display(thinking)

                    response = self.rag_chain.invoke({"input": formatted_prompt})
                    thinking.value = ""
                    print(f"{response['answer']}\n\n")

                    self.name.disabled = True
                    self.ticket_id.disabled = True
                    self.btn.disabled = True
                    self.name = None
                    self.ticket_id = None
                    self.display_box()
                else:
                    print("Invalid JIRA ticket id!")
            else:
                print("Please enter the component name and JIRA ticket ID")

# Initialize chat model
chat_model = ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0",
    client=bedrock_client,
    model_kwargs={
        "temperature": 0.2,
        "topP": 0.1,
    },
)

# Define system prompt
system_prompt = (
    "You are an assistant for generating Angular components. "
    "Generate the component code using the following context:"
    "\\n\\n"
    "{context}"
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create retrieval chain
retriever = vectorstore_faiss.as_retriever(search_kwargs={"k": 6})
question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Generate the prompt template
prompt_template = (
    "Create a component named {}.\n"
    "Use the 'Component structure' guidelines provided\n"
    "Component should also follow these requirements: \n\n```{}```\n\n"
)

# Pass the prompt template to ChatUX
chat = ChatUX(
    rag_chain=rag_chain, 
    prompt_template=prompt_template, 
    jira_lookup_tool=jira_lookup_tool
)
chat.start_chat()

## Conclusion

Through adequate tuning, the model successfully generated code that adheres to the established boilerplate. This approach can be seamlessly extended to various types of codebases, including web, mobile, APIs, and more.